# Bring it all together
Creates a single NetCDF (.nc) file for each gas species

In [1]:
from thermotools.plot import *

import netCDF4 as nc
import time, glob, os
import numpy as np

from thermotools import moles
from thermotools import get_gendir, get_inpdir, empty_dir
from thermotools.util import makezip

fbig = 1e90
today = str(time.strftime("%Y-%m-%d"))
print(today)

2025-02-17


In [2]:
datdir = os.path.join(get_gendir(), "compiled", "dat")
empty_dir(datdir)

pltdir = os.path.join(get_gendir(), "compiled", "plt")
empty_dir(pltdir)

In [3]:
gases = ['H','C','N','O','S','Fe','Mg','Si','Na','He',
         'O2', 'O3', 'N2', 'N2O', 'NO', 'CH3', 'H2O2',
         'SO2', 'NH3', 'H2', 'CO2', 'H2O', 'CO', 'HCl',
         'HCN', 'CH4', 'H2SO4', 'H2S', 'OH', 'SiO', 'VO',
         'SiO2', 'FeO2', 'SO', 'S2', 'S8', 'OCS', 'TiO',
         'FeH', 'C2H4', 'C2H6', 'HNO3', 'Fe', 'PH3',
         'FeO','Na2','NaO','Mg','Mg2','MgO', 'CH3', 'SF6']

gases = list(set(gases))

In [4]:
elem_table = moles.read_elements()

In [5]:
# Find the name JANAF gives 'gas', based on number of atoms
def find_janaf_name(gas:str):

    target = moles.count_atoms(gas)

    files = glob.glob(os.path.join(get_gendir(),"cp","dat")+"/*.csv")
    for f in files:
        form  = moles.formula_from_path(f)
        atoms = moles.count_atoms(form)

        good = True

        for k in atoms.keys():
            if k not in target.keys():
                good = False
                break

        for k in target.keys():
            if k not in atoms.keys():
                good = False
                break
            if target[k] != atoms[k]:
                good = False
                break

        if good:
            return form

    return ""


In [6]:
def compile(gas:str):
    print("Compiling %s"%gas)

    # JANAF alias
    janaf = find_janaf_name(gas)
    if len(janaf) == 0:
        print("    skipping (no JANAF data)")
        return

    # MMW calculation
    mmw_val = moles.mmw_from_formula(gas, elem_table)

    # create dataset
    fpath = datdir+"/%s.nc"%gas
    if os.path.exists(fpath):
        os.remove(fpath)
    ds = nc.Dataset(fpath, 'w')

    # JANAF name
    ds.createDimension('njanaf', len(janaf))
    NC_janaf = ds.createVariable('JANAF', 'S1', ('njanaf'))
    NC_janaf[:] = nc.stringtochar(np.array([janaf], 'S'))

    # Date created
    ds.createDimension('ndate', len(today))
    NC_today = ds.createVariable('created', 'S1', ('ndate'))
    NC_today[:] = nc.stringtochar(np.array([today], 'S'))

    # MMW
    NC_mmw = ds.createVariable("mmw","f8")
    NC_mmw[:] = mmw_val
    NC_mmw.units = "kg mol-1"

    # Saturation curve
    satdat = os.path.join(get_gendir(),"sat","dat")
    sat_path = satdat+"/%s_sat.csv"%gas
    if os.path.exists(sat_path):
        # Critical point
        T_crit = float(np.loadtxt(satdat+"/%s_crit.csv"%gas))
        NC_tcrit = ds.createVariable("T_crit","f8")
        NC_tcrit[:] = T_crit
        NC_tcrit.units = "K"

        # Triple point
        T_trip = float(np.loadtxt(satdat+"/%s_trip.csv"%gas))
        NC_ttrip = ds.createVariable("T_trip","f8")
        NC_ttrip[:] = T_trip
        NC_ttrip.units = "K"

        # Saturation curve
        X_sat  = np.loadtxt(sat_path, delimiter=',').T
        ds.createDimension("sat",len(X_sat[0]))
        #    temperatures
        NC_tsat = ds.createVariable("sat_T","f8","sat")
        NC_tsat[:] = X_sat[0][:]
        NC_tsat.units = "K"
        #    pressures
        NC_psat = ds.createVariable("sat_P","f8","sat")
        NC_psat[:] = X_sat[1][:]
        NC_psat.units = "Pa"
    else:
        print("    without saturation curve")


    # Latent heat
    lat_path = os.path.join(get_gendir(),"lv","dat")+"/%s.csv"%gas
    if os.path.exists(lat_path):
        X_lat = np.loadtxt(lat_path, delimiter=',').T
        ds.createDimension("lat",len(X_lat[0]))
        #    lookup temperatures
        NC_dHt = ds.createVariable("lat_T","f8","lat")
        NC_dHt[:] = X_lat[0][:]
        NC_dHt.units = "K"
        #    enthalpy changes [J/kg]
        NC_dH = ds.createVariable("lat_H","f8","lat")
        NC_dH[:] = X_lat[1][:]
        NC_dH.units = "J kg-1"
    else:
        print("    without latent heat")


    # Heat capacity
    cap_path = os.path.join(get_gendir(),"cp","dat")+"/%s.csv"%janaf
    X_cap = np.loadtxt(cap_path, delimiter=',').T
    ds.createDimension("cap",len(X_cap[0]))
    #    lookup temperatures
    NC_cpt = ds.createVariable("cap_T","f8","cap")
    NC_cpt[:] = X_cap[0][:]
    NC_cpt.units = "K"
    #   heat capacity
    NC_cp = ds.createVariable("cap_C","f8","cap")
    NC_cp[:] = X_cap[1][:] / mmw_val # convert from "per mol" to "per kg"
    NC_cp.units = "J K-1 kg-1"

    # Van der Waals equation of state (density vs T,P)
    vdw_path = os.path.join(get_gendir(),"vdw","dat")+"/%s.csv"%gas
    if os.path.exists(vdw_path):
        # Load data
        X_vdw = np.loadtxt(vdw_path, delimiter=',').T
        T_vdw = np.unique(X_vdw[0][:])
        P_vdw = np.unique(X_vdw[1][:])

        #    lookup temperatures
        ds.createDimension("vdw_T",len(T_vdw))
        NC_vdwT = ds.createVariable("vdw_T","f8","vdw_T")
        NC_vdwT[:] = T_vdw[:]
        NC_vdwT.units = "K"
        #    lookup pressures [log Pa]
        ds.createDimension("vdw_P",len(P_vdw))
        NC_vdwP = ds.createVariable("vdw_P","f8","vdw_P")
        NC_vdwP[:] = P_vdw[:]
        NC_vdwP.units = "log10 Pa"
        #    density [log kg m-3]
        NC_vdwR = ds.createVariable("vdw_rho","f8",("vdw_P","vdw_T"))
        NC_vdwR[:] = np.reshape(X_vdw[2][:], (len(P_vdw), len(T_vdw)))
        NC_vdwR.units = "log10 kg m-3"
    else:
        print("    without VdW EOS")

    # AQUA equation of state (density vs T,P)
    if gas == "H2O":
        fpath = os.path.join(get_gendir(),"aqua","dat","H2O.csv")
        if os.path.exists(fpath):
            # Load data
            X_eos = np.loadtxt(fpath, delimiter=',').T
            T_eos = np.unique(X_eos[0][:])
            P_eos = np.unique(X_eos[1][:])

            #    lookup temperatures
            ds.createDimension("aqua_T",len(T_eos))
            NC_eosT = ds.createVariable("aqua_T","f8","aqua_T")
            NC_eosT[:] = T_eos[:]
            NC_eosT.units = "K"
            #    lookup pressures [log Pa]
            ds.createDimension("aqua_P",len(P_eos))
            NC_eosP = ds.createVariable("aqua_P","f8","aqua_P")
            NC_eosP[:] = P_eos[:]
            NC_eosP.units = "log10 Pa"
            #    density [log kg m-3]
            NC_eosR = ds.createVariable("aqua_rho","f8",("aqua_P","aqua_T"))
            NC_eosR[:] = np.reshape(X_eos[2][:], (len(P_eos), len(T_eos)))
            NC_eosR.units = "log10 kg m-3"
        else:
            print("    without AQUA EOS")

    # CMS19 equation of state (density vs T,P)
    if gas == "H2":
        fpath = os.path.join(get_gendir(),"cms19","dat","H2.csv")
        if os.path.exists(fpath):
            # Load data
            X_eos = np.loadtxt(fpath, delimiter=',').T
            T_eos = np.unique(X_eos[0][:])
            P_eos = np.unique(X_eos[1][:])

            #    lookup temperatures
            ds.createDimension("cms19_T",len(T_eos))
            NC_eosT = ds.createVariable("cms19_T","f8","cms19_T")
            NC_eosT[:] = T_eos[:]
            NC_eosT.units = "K"
            #    lookup pressures [log Pa]
            ds.createDimension("cms19_P",len(P_eos))
            NC_eosP = ds.createVariable("cms19_P","f8","cms19_P")
            NC_eosP[:] = P_eos[:]
            NC_eosP.units = "log10 Pa"
            #    density [log kg m-3]
            NC_eosR = ds.createVariable("cms19_rho","f8",("cms19_P","cms19_T"))
            NC_eosR[:] = np.reshape(X_eos[2][:], (len(P_eos), len(T_eos)))
            NC_eosR.units = "log10 kg m-3"
        else:
            print("    without CMS19 EOS")

    # done
    ds.close()


In [7]:
print("Compiling individual data files...")
for g in gases:
    compile(g)

print(" ")
files = glob.glob(datdir+"/*.nc")
files = [os.path.abspath(f) for f in files]
zpath = os.path.join(datdir, "gases.zip")
makezip(zpath, files)

Compiling individual data files...
Compiling MgO
    without saturation curve
    without latent heat
    without VdW EOS
Compiling C2H4
    without saturation curve
    without VdW EOS
Compiling H2
Compiling Si
    without saturation curve
    without latent heat
    without VdW EOS
Compiling FeH
    skipping (no JANAF data)
Compiling O3
    without saturation curve
    without latent heat
Compiling Mg2
    without saturation curve
    without latent heat
    without VdW EOS
Compiling NH3
Compiling Fe
    without saturation curve
    without latent heat
    without VdW EOS
Compiling O
    without saturation curve
    without latent heat
    without VdW EOS
Compiling CO
    without saturation curve
Compiling H2O2
    without saturation curve
    without latent heat
    without VdW EOS
Compiling NO
    without saturation curve
    without latent heat
Compiling SiO
    without saturation curve
    without latent heat
    without VdW EOS
Compiling S8
    without saturation curve
    witho

In [8]:
def no_data_text(ax):
    ax.text(0.5, 0.5, "NO DATA", transform=ax.transAxes, horizontalalignment='center', verticalalignment='center', color='tab:red')

In [9]:
# Make plots
print("Making plots from NetCDF files...")

files = glob.glob(datdir+"/*.nc")
print("Found %d files"%len(files))

for f in files:

    gas = moles.formula_from_path(f)
    print("    "+gas)

    ds = nc.Dataset(f,'r')

    fig,axs = plt.subplots(4,1, sharex=True, figsize=(9,8))

    axs[0].set_title(gas)

    axs[0].set_ylabel(r"P$_{\text{sat}}$" +"\n[bar]")
    if "sat_T" in ds.variables.keys():
        axs[0].plot(ds.variables["sat_T"][:],  ds.variables["sat_P"][:]*1e-5)
        axs[0].set_yscale("symlog", linthresh=1e-4)
    else:
        no_data_text(axs[0])

    axs[1].set_ylabel(r"$\Delta$H"+"\n"+r"[kJ kg$^{-1}$]")
    if "lat_T" in ds.variables.keys():
        axs[1].plot(ds.variables["lat_T"][:],    ds.variables["lat_H"][:]*1e-3)
    else:
        no_data_text(axs[1])

    axs[2].set_ylabel(r"$C_p$" + "\n" + r"[kJ K$^{-1}$ kg$^{-1}$]")
    if "cap_T" in ds.variables.keys():
        axs[2].plot(ds.variables["cap_T"][:],    ds.variables["cap_C"][:]*1e-3)
    else:
        no_data_text(axs[2])

    has_eos = False
    axs[3].set_ylabel("Gas density @ 1 bar \n" + r"[kg m$^{-3}$]")
    if "vdw_T" in ds.variables.keys():
        arr_t = ds.variables["vdw_T"][:]
        arr_p = ds.variables["vdw_P"][:]
        arr_r = ds.variables["vdw_rho"][:]
        iclose = np.argmin(np.abs(arr_p - 5))
        axs[3].plot(arr_t,10**arr_r[iclose,:], label="Van der Waals")
        has_eos = True
    if "aqua_T" in ds.variables.keys():
        arr_t = ds.variables["aqua_T"][:]
        arr_p = ds.variables["aqua_P"][:]
        arr_r = ds.variables["aqua_rho"][:]
        iclose = np.argmin(np.abs(arr_p - 5))
        axs[3].plot(arr_t,10**arr_r[iclose,:], label="AQUA", ls='dashed')
        has_eos = True
    if has_eos:
        axs[3].legend()
        axs[3].set_yscale("log")
    else:
        no_data_text(axs[3])


    axs[-1].set_xlabel("Temperature [K]")
    fig.savefig(pltdir + "/%s.png"%gas, bbox_inches='tight', dpi=220)
    plt.close("all")

    ds.close()
print("Done!")

Making plots from NetCDF files...
Found 40 files
    MgO
    C2H4
    H2
    Si
    O3
    Mg2
    NH3
    Fe
    O
    CO
    H2O2
    NO
    SiO
    S8
    VO
    OCS
    CH4
    Mg
    S
    N2O
    O2
    H2S
    OH
    H2SO4
    S2
    N
    CH3
    N2
    CO2
    HCN
    C
    SO
    PH3
    SiO2
    SO2
    TiO
    HNO3
    FeO
    H
    H2O
Done!
